In [1]:
#1. 为了方便调试，先在AWS sagemaker的notebook实例上对小数据集进行训练，部署和预测。

In [ ]:
#A. 构建在notebook实例上训练的环境

In [51]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json    
!/bin/bash ./local_mode_setup.sh

SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [2]:
!pwd

/home/ec2-user/SageMaker/SM test code/deepfm-ctr


In [ ]:
#B. 在notebook实例上进行训练，注意model_dir，servable_model_dir必须设置为/opt/ml/model，否则之后部署模型会报错。
#下面超参数的设置请参考https://github.com/lambdaji/tf_repos.git中的说明。这里为了快速测试，epoch设置为1.

In [51]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

model_dir = '/opt/ml/model'
train_instance_type = 'local'
checkpoint_s3_uri = 's3://liang200/deepfm-checkpoint'
checkpoint_local_path = '/opt/ml/checkpoints',
distributions={'parameter_server': {'enabled': True}}

#设置python程序中的参数checkpoinPath与tensorflow estimator设置的checkpoint_local_path一致
hyperparameters = {'servable_model_dir': '/opt/ml/model', 'data_dir': '/opt/ml/input/data/training/',
                'checkpoinPath': '/opt/ml/checkpoints', 'log_steps': 10, 'num_epochs': 1, 'field_size': 39, 'feature_size': 117581, 'deep_layers': '2,2,2'
                  }
local_estimator = TensorFlow(
                       #source_dir='tf_repos/deep_ctr/Model_pipeline',
                       entry_point='DeepFM-mirrorstrategy-singleinstanceMutipleCPU.py',
                       #model_dir=model_dir,
                       #checkpoint_path = 's3://liang200', #just for test 
                       #checkpoint_s3_uri = 's3://liang200/deepfm-checkpoint',
                       #checkpoint_local_path = '/opt/ml/checkpoints/',
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-scriptmode-deepctr-deepfm',
                       framework_version='1.14',
                       #distributions=distributions,
                       py_version='py3')


In [52]:
"""
import os
train_dir = os.path.join(os.getcwd(), 'raw')
inputs = {'training': f'file://{train_dir}'}
train_dir

local_estimator.fit(inputs)
"""


train_s3 = 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode-training-data'
validate_s3 = 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode-evaluation-data'
inputs = {'training':train_s3, 'evaluation': validate_s3}
print(inputs)
local_estimator.fit(inputs)

{'training': 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode-training-data', 'evaluation': 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode-evaluation-data'}
Creating tmpzz_2baws_algo-1-1fakf_1 ... 
Attaching to tmpzz_2baws_algo-1-1fakf_12mdone
algo-1-1fakf_1  | /usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
algo-1-1fakf_1  |   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
algo-1-1fakf_1  | /usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
algo-1-1fakf_1  |   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
algo-1-1fakf_1  | /usr/local/lib/python3.6/site-package

KeyboardInterrupt: 

In [ ]:
#C. 在notebook实例上部署，设置instance_type为local

In [54]:
local_predictor = local_estimator.deploy(initial_instance_count=1,instance_type='local')

Attaching to tmpszqjxs73_algo-1-foa5s_1
algo-1-foa5s_1  | INFO:__main__:starting services
algo-1-foa5s_1  | INFO:__main__:using default model name: model
algo-1-foa5s_1  | INFO:__main__:tensorflow serving model config: 
algo-1-foa5s_1  | model_config_list: {
algo-1-foa5s_1  |   config: {
algo-1-foa5s_1  |     name: "model",
algo-1-foa5s_1  |     base_path: "/opt/ml/model",
algo-1-foa5s_1  |     model_platform: "tensorflow"
algo-1-foa5s_1  |   },
algo-1-foa5s_1  | }
algo-1-foa5s_1  | 
algo-1-foa5s_1  | 
algo-1-foa5s_1  | INFO:__main__:nginx config: 
algo-1-foa5s_1  | load_module modules/ngx_http_js_module.so;
algo-1-foa5s_1  | 
algo-1-foa5s_1  | worker_processes auto;
algo-1-foa5s_1  | daemon off;
algo-1-foa5s_1  | pid /tmp/nginx.pid;
algo-1-foa5s_1  | error_log  /dev/stderr info;
algo-1-foa5s_1  | 
algo-1-foa5s_1  | worker_rlimit_nofile 4096;
algo-1-foa5s_1  | 
algo-1-foa5s_1  | events {
algo-1-foa5s_1  |   worker_connections 2048;
algo-1-foa5s_1  | }
algo-1-foa5s_1  | 
algo-1-foa5s_1 

In [ ]:
#D. 在notebook上部署的模型做预测，注意这里的输入数据的格式要匹配到deepfm.py中的feature spec。

In [55]:
test_example = {'feat_ids': [1,2,3,4,5,6,7,8,9,10,11,12,13,15,555,1078,17797,26190,26341,28570,35361,35613,35984,48424,51364,64053,65964,66206,71628,84088,84119,86889,88280,88283,100288,100300,102447,109932,111823],
                'feat_vals': [0.05,0.006633,0.1,0,0.021594,0.008,0.15,0.04,0.362,0.1,0.2,0,0.04,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
                }
local_results = local_predictor.predict(test_example)

algo-1-foa5s_1  | 172.18.0.1 - - [16/Feb/2020:09:41:54 +0000] "POST /invocations HTTP/1.1" 200 38 "-" "-"


In [56]:
local_results

{'predictions': [0.296024]}

In [57]:
#E. 删除notebook实例上的模型部署。
local_predictor.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)


In [ ]:
#2. 在上面测试成功后，就可以用Sagemaker hosted的方式来训练，部署。
#Sagemaker hosted training

In [ ]:
#A. 把数据集上传到S3

In [23]:
s3_prefix = 'tf-SM-deepctr-deepfm'

traindata_s3_prefix = '{}/data'.format(s3_prefix)


In [2]:
#train_s3 = sagemaker.Session().upload_data(path='raw', key_prefix=traindata_s3_prefix)
train_s3= 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/data'


In [42]:
inputs = {'training':train_s3}

print(inputs)


{'training': 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/data'}


In [ ]:
#B. 这里用GPU实例来训练，超参数需要修改。这里为了测试方便，没有对超参数做修改。

In [49]:
#下面用多个spot实例进行parameter server方式的分布式训练。
import sagemaker
#from sagemaker.tensorflow import TensorFlow
from sagemaker.tensorflow.estimator import TensorFlow

import os

#train_instance_type = 'ml.m5.24xlarge'
train_instance_type = 'ml.c5.18xlarge'
#train_instance_type = 'ml.g4dn.12xlarge'
distributions={'parameter_server': {'enabled': True}}
train_use_spot_instances = True
#train_use_spot_instances = False

train_max_run=36000*2
train_max_wait = 72000 if train_use_spot_instances else None
model_dir = 's3://test-s3-delete/deepfm-multiCPU-testliang-testexit24const12345623native/'
#model_dir = '/opt/ml/model'
#deep_layer = '4096,4096,4096'   #这个对应checkpoint大概500MB，没有问题
#deep_layer = '8192,4096,4096'   #这个对应checkpoint大概800MB，4台c5.18xlarge没有问题，2台c5.9xlarge/p3.16xlarge都有问题
#deep_layer = '8192,8192,4096'   #这个对应checkpoint大概1300MB，有问题
#deep_layer = '128,64,32'
deep_layer = '2,2,2'

batch_size = 1
feature_size = 1117581*4
#feature_size = 91758100
#feature_size = 51758100
#feature_size = 1000000000
#feature_size = 500000000
#feature_size = 100000000
#feature_size = 51758100

#设置python程序中的参数checkpoinPath与tensorflow estimator设置的checkpoint_local_path一致
hyperparameters = {'servable_model_dir': '/opt/ml/model', 'data_dir': '/opt/ml/input/data/training/',
                'checkpoinPath': '/home/checkpoints', 'log_steps': 10, 'num_epochs': 2, 'field_size': 39, 'feature_size': feature_size, 'deep_layers': deep_layer,
                   'perform_shuffle': 0, 'batch_size': batch_size, 'pipe_mode': 1
                  }
estimator = TensorFlow(
                       source_dir='test_S3',
                       entry_point='shell_launch.sh',
                       #entry_point='DeepFM-dist-ps-for-multiGPU-multiInstance.py',
                       #entry_point='DeepFM-mirrorstrategy-singleinstanceMutipleGPU.py',
                       model_dir=model_dir,
                       #checkpoint_s3_uri = 's3://liang200/deepfm-checkpoint',
                       #checkpoint_local_path = '/home/checkpoints',
                       output_path= 's3://liang200/deepfm-model',
                       train_instance_type=train_instance_type,
                       train_instance_count=4,
                       train_volume_size = 500,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-scriptmode-deepctr-deepfm',
                       framework_version='1.14',
                       #image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.2-cpu-py3",
                       py_version='py3',
                       script_mode=True,
                       input_mode='Pipe',
                       #debugger_hook_config = False,
                       distributions=distributions,
                       train_use_spot_instances=train_use_spot_instances,
                    
                       train_max_wait=train_max_wait,
                       train_max_run=train_max_run
                       )

Parameter distribution will be renamed to {'parameter_server': {'enabled': True}} in SageMaker Python SDK v2.


In [2]:
!pwd

/home/ec2-user/SageMaker/deepfm test


In [50]:
#下面这个测试pipe mode
train_s3 = 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode-evaluation-data'
validate_s3 = 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode-evaluation-data'
inputs = {'training':train_s3, 'evaluation': validate_s3}
print(inputs)
estimator.fit(inputs)

"""
#下面这个测试file mode
#train_s3 = 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/filemode_data'
train_s3 = 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode-training-data'

inputs = {'training':train_s3}
print(inputs)
estimator.fit(inputs)
"""

#test for tfrecord on file mode
"""
train_s3 = 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/tfrecords'
inputs = {'training':train_s3}
print(inputs)
estimator.fit(inputs)


#test for tfrecord on pipe mode
train_s3 = 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode_tfrecord_train'
validate_s3 = 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode_tfrecord_evaluation'
inputs = {'training':train_s3, 'evaluation': validate_s3}
print(inputs)
estimator.fit(inputs)
"""

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


{'training': 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode-evaluation-data', 'evaluation': 's3://sagemaker-us-east-1-514385905925/tf-SM-deepctr-deepfm/pipemode-evaluation-data'}
2020-11-27 04:01:21 Starting - Starting the training job...
2020-11-27 04:01:23 Starting - Launching requested ML instances.........
2020-11-27 04:02:54 Starting - Preparing the instances for training......
2020-11-27 04:04:14 Downloading - Downloading input data
2020-11-27 04:04:14 Training - Downloading the training image../usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future versio

UnexpectedStatusException: Error for Training job tf-scriptmode-deepctr-deepfm-2020-11-27-04-01-21-517: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/bin/sh -c ./shell_launch.sh --batch_size 1 --checkpoinPath /home/checkpoints --data_dir /opt/ml/input/data/training/ --deep_layers 2,2,2 --feature_size 4470324 --field_size 39 --log_steps 10 --model_dir s3://test-s3-delete/deepfm-multiCPU-testliang-testexit24const12345623native/ --num_epochs 2 --perform_shuffle 0 --pipe_mode 1 --servable_model_dir /opt/ml/model"

In [10]:
test_config = {'cluster': {'master': ['algo-1:2222'], 'ps': ['algo-1:2223', 'algo-2:2223'], 'worker': ['algo-2:2222']}, 'environment': 'cloud', 'task': {'index': 0, 'type': 'worker'}}

In [13]:
if test_config :
    print("enter")
    os.environ['test_config'] = test_config.replace('master', 'chief')

enter


AttributeError: 'dict' object has no attribute 'replace'

In [12]:
test = os.environ.get('test_config')
test

In [14]:
#查看模型的保存位置。
estimator.model_data

's3://liang200/deepfm-model/tf-scriptmode-deepctr-deepfm-2020-03-12-04-18-09-413/output/model.tar.gz'

In [78]:
#C. 用Sagemaker的GPU实例来部署模型。

In [15]:
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.c5.2xlarge')

-----------!

In [ ]:
#D. 用单个样本来预测

In [16]:
#这个样本的label是0
test_example = {'feat_ids': [1,2,3,4,5,6,7,8,9,10,11,12,13,15,555,1078,17797,26190,26341,28570,35361,35613,35984,48424,51364,64053,65964,66206,71628,84088,84119,86889,88280,88283,100288,100300,102447,109932,111823],
                'feat_vals': [0.05,0.006633,0.05,0,0.021594,0.008,0.15,0.04,0.362,0.1,0.2,0,0.04,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
                }
results = predictor.predict(test_example)
results

{'predictions': [0.338876218]}

In [81]:
#查看部署的模型的endpoint名字
predictor.endpoint

'tf-scriptmode-deepctr-deepfm-2020-02-16-14-59-46-265'

In [ ]:
#3. 如果想在Sagemaker外面来进行API call做预测的话，参考如下的代码

In [82]:
import os
import io
import boto3
import json
import csv

endpointname = predictor.endpoint
runtime= boto3.client('runtime.sagemaker')

test_example = {'feat_ids': [1,2,3,4,5,6,7,8,9,10,11,12,13,15,555,1078,17797,26190,26341,28570,35361,35613,35984,48424,51364,64053,65964,66206,71628,84088,84119,86889,88280,88283,100288,100300,102447,109932,111823],
                'feat_vals': [0.05,0.006633,0.05,0,0.021594,0.008,0.15,0.04,0.362,0.1,0.2,0,0.04,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
                }

print(test_example)

response = runtime.invoke_endpoint(EndpointName=endpointname,
                                   ContentType='application/json',
                                   Body=json.dumps(test_example))
print(response)
result = json.loads(response['Body'].read().decode())
print(result)


{'feat_ids': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 555, 1078, 17797, 26190, 26341, 28570, 35361, 35613, 35984, 48424, 51364, 64053, 65964, 66206, 71628, 84088, 84119, 86889, 88280, 88283, 100288, 100300, 102447, 109932, 111823], 'feat_vals': [0.05, 0.006633, 0.05, 0, 0.021594, 0.008, 0.15, 0.04, 0.362, 0.1, 0.2, 0, 0.04, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'ResponseMetadata': {'RequestId': '0d66cdcc-0c19-45d4-b2b3-7b411b0ae581', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0d66cdcc-0c19-45d4-b2b3-7b411b0ae581', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Sun, 16 Feb 2020 15:42:51 GMT', 'content-type': 'application/json', 'content-length': '39'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f37021184e0>}
{'predictions': [0.0264763]}


In [ ]:
#4. 做完实验，删掉部署的endpoint防止产生不必要的费用。
sagemaker.Session().delete_endpoint(predictor.endpoint)

In [1]:
%%script bash
export S3_MULTI_PART_UPLOAD_CHUNK_SIZE=500 * 1024 * 1024

bash: line 1: export: `daemon.json': not a valid identifier
bash: line 1: export: `DeepFM-dist-ps-for-multiGPU-multiInstance.py': not a valid identifier
bash: line 1: export: `DeepFM-dist-ps-for-multipleCPU-multiInstance.py': not a valid identifier
bash: line 1: export: `DeepFM-hvd.py': not a valid identifier
bash: line 1: export: `DeepFM-hvd-tfrecord-vectorized-map.py': not a valid identifier
bash: line 1: export: `DeepFM-mirrorstrategy-singleinstanceMutipleCPU.py': not a valid identifier
bash: line 1: export: `DeepFM-mirrorstrategy-singleinstanceMutipleGPU.py': not a valid identifier
bash: line 1: export: `DeepFM.py': not a valid identifier
bash: line 1: export: `deepfm-sagemaker-helper-hvd.ipynb': not a valid identifier
bash: line 1: export: `deepfm-sagemaker-helper-non_hvd.ipynb': not a valid identifier
bash: line 1: export: `DeepFM-tfrecord-vetorized_map-for-CPU.py': not a valid identifier
bash: line 1: export: `DeepFM-tfrecord-vetorized_map-for-GPU-Copy1.py': not a valid identifi

CalledProcessError: Command 'b'export S3_MULTI_PART_UPLOAD_CHUNK_SIZE=500 * 1024 * 1024\n'' returned non-zero exit status 1.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=''

import tensorflow as tf
import numpy as np

print(tf.__version__)
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("L_enc", [4000000, 1023], initializer=tf.glorot_normal_initializer())
v2 = tf.get_variable("L_dec", [4000000, 1023], initializer=tf.glorot_normal_initializer())
v1_plhdr = tf.placeholder(dtype=tf.float32, shape=[4000000, 1023])

# Add an op to initialize the variables.
init_op = tf.initialize_all_variables()

# Add ops to save and restore all the variables.
saver = tf.train.Saver(tf.all_variables())

embedding = np.random.uniform(-1, 1, (4000000, 1023))

with tf.Session() as sess:
  sess.run(init_op)
  for v in tf.trainable_variables():
    sess.run(v.assign(v1_plhdr), {v1_plhdr: embedding})
    #sess.run(v.assign(embedding))
  print("finish")
  # Save the variables to disk.
  save_path = saver.save(sess, "./model.ckpt")
  print("Model saved in file: %s" % save_path)

1.15.3
finish


In [1]:
print("finish")

finish


In [9]:
!ls -al 

total 32001016
drwxrwxr-x  8 ec2-user ec2-user        4096 Nov 23 07:09 .
drwxr-xr-x 93 ec2-user ec2-user        4096 Nov  5 07:37 ..
-rw-rw-r--  1 ec2-user ec2-user          77 Nov 23 07:09 checkpoint
-rw-rw-r--  1 ec2-user ec2-user         176 Sep 27 03:42 daemon.json
-rw-rw-r--  1 ec2-user ec2-user       24336 Nov 18 02:49 DeepFM-dist-ps-for-multiGPU-multiInstance.py
-rw-rw-r--  1 ec2-user ec2-user       26027 Nov 18 02:49 DeepFM-dist-ps-for-multipleCPU-multiInstance.py
-rw-rw-r--  1 ec2-user ec2-user       26855 Nov 18 02:49 DeepFM-hvd.py
-rw-rw-r--  1 ec2-user ec2-user       27088 Nov 18 02:49 DeepFM-hvd-tfrecord-vectorized-map.py
-rw-rw-r--  1 ec2-user ec2-user       24126 Aug 26 15:08 DeepFM-mirrorstrategy-singleinstanceMutipleCPU.py
-rw-rw-r--  1 ec2-user ec2-user       23855 Oct 19 10:32 DeepFM-mirrorstrategy-singleinstanceMutipleGPU.py
-rw-rw-r--  1 ec2-user ec2-user       22833 Aug 26 15:08 DeepFM.py
-rw-rw-r--  1 ec2-user ec2-user      236264 Nov 18 02:49 deepfm-sagemaker-h

In [2]:
test = np.random.uniform(-1, 1, (4000000, 1024))
test.shape

(4000000, 1024)

In [4]:
w_init = np.random.randn(2**22, 256).astype(np.float32)
w = tf.Variable(tf.convert_to_tensor(w_init))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.reduce_sum(w)))

ValueError: Cannot create a tensor proto whose content is larger than 2GB.

In [6]:

w_init = np.random.randn(2**22, 256).astype(np.float32)
w_plhdr = tf.placeholder(dtype=tf.float32, shape=[2**22, 256])
w = tf.get_variable('w', [2**22, 256])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(w.assign(w_plhdr), {w_plhdr: w_init})
    print(sess.run(tf.reduce_sum(w)))

94744.836


In [ ]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("L_enc", [400000, 1024])
v2 = tf.get_variable("L_dec", [400000, 1024])

# Add an op to initialize the variables.
init_op = tf.initialize_all_variables()

# Add ops to save and restore all the variables.
saver = tf.train.Saver(tf.all_variables())

with tf.Session() as sess:
  sess.run(init_op)
  for v in tf.trainable_variables():
    embedding = np.random.uniform(-1, 1, (400000, 1024))
    sess.run(v.assign(embedding))
  print("finish")
  # Save the variables to disk.
  save_path = saver.save(sess, "./model.ckpt")
  print("Model saved in file: %s" % save_path)

finish


In [2]:
!ls -al

total 3201012
drwxrwxr-x  8 ec2-user ec2-user       4096 Nov 24 02:40 .
drwxr-xr-x 93 ec2-user ec2-user       4096 Nov  5 07:37 ..
-rw-rw-r--  1 ec2-user ec2-user         77 Nov 23 07:44 checkpoint
-rw-rw-r--  1 ec2-user ec2-user        176 Sep 27 03:42 daemon.json
-rw-rw-r--  1 ec2-user ec2-user      24336 Nov 18 02:49 DeepFM-dist-ps-for-multiGPU-multiInstance.py
-rw-rw-r--  1 ec2-user ec2-user      26027 Nov 18 02:49 DeepFM-dist-ps-for-multipleCPU-multiInstance.py
-rw-rw-r--  1 ec2-user ec2-user      26855 Nov 18 02:49 DeepFM-hvd.py
-rw-rw-r--  1 ec2-user ec2-user      27088 Nov 18 02:49 DeepFM-hvd-tfrecord-vectorized-map.py
-rw-rw-r--  1 ec2-user ec2-user      24126 Aug 26 15:08 DeepFM-mirrorstrategy-singleinstanceMutipleCPU.py
-rw-rw-r--  1 ec2-user ec2-user      23855 Oct 19 10:32 DeepFM-mirrorstrategy-singleinstanceMutipleGPU.py
-rw-rw-r--  1 ec2-user ec2-user      22833 Aug 26 15:08 DeepFM.py
-rw-rw-r--  1 ec2-user ec2-user     236264 Nov 18 02:49 deepfm-sagemaker-helper-hvd.ipy

In [3]:
!ls -al

total 3201016
drwxrwxr-x  8 ec2-user ec2-user       4096 Nov 24 02:42 .
drwxr-xr-x 93 ec2-user ec2-user       4096 Nov  5 07:37 ..
-rw-rw-r--  1 ec2-user ec2-user         77 Nov 23 07:44 checkpoint
-rw-rw-r--  1 ec2-user ec2-user        176 Sep 27 03:42 daemon.json
-rw-rw-r--  1 ec2-user ec2-user      24336 Nov 18 02:49 DeepFM-dist-ps-for-multiGPU-multiInstance.py
-rw-rw-r--  1 ec2-user ec2-user      26027 Nov 18 02:49 DeepFM-dist-ps-for-multipleCPU-multiInstance.py
-rw-rw-r--  1 ec2-user ec2-user      26855 Nov 18 02:49 DeepFM-hvd.py
-rw-rw-r--  1 ec2-user ec2-user      27088 Nov 18 02:49 DeepFM-hvd-tfrecord-vectorized-map.py
-rw-rw-r--  1 ec2-user ec2-user      24126 Aug 26 15:08 DeepFM-mirrorstrategy-singleinstanceMutipleCPU.py
-rw-rw-r--  1 ec2-user ec2-user      23855 Oct 19 10:32 DeepFM-mirrorstrategy-singleinstanceMutipleGPU.py
-rw-rw-r--  1 ec2-user ec2-user      22833 Aug 26 15:08 DeepFM.py
-rw-rw-r--  1 ec2-user ec2-user     236264 Nov 18 02:49 deepfm-sagemaker-helper-hvd.ipy

In [6]:
import tensorflow as tf
var_list = tf.train.list_variables("./model.ckpt")
var_list

[('L_dec', [400000, 1024]), ('L_enc', [400000, 1024])]

In [11]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph 


# 模型的checkpoint文件地址
ckpt_path = "./model.ckpt"

tf.reset_default_graph() 

with tf.Session() as sess:
    #保存图，在./pb_model文件夹中生成model.pb文件
    # model.pb文件将作为input_graph给到接下来的freeze_graph函数
    tf.train.write_graph(sess.graph_def, './pb_model', 'model.pb') 
    #把图和参数结构一起 
    freeze_graph.freeze_graph(
        input_graph='./pb_model/model.pb',
        input_saver='',
        input_binary=False, 
        input_checkpoint=ckpt_path, 
        output_node_names='out',
        restore_op_name='',
        filename_tensor_name='',
        variable_names_blacklist = "L_dec",
        output_graph='./pb_model/frozen_model.pb',
        clear_devices=False,
        initializer_nodes=''
        )

print("done") 


ValueError: No variables to save

In [4]:
import sagemaker
print(sagemaker.__version__)

1.72.1


In [17]:
feature_size = 204
test = feature_size//2
test

102

In [40]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph() 

value = [0, 1, 2, 3, 4, 5, 6, 7]
value = np.array(value)
# value = value.reshape([2, 4])
init = tf.constant_initializer(value)

print('fitting shape:')
tf.reset_default_graph()
with tf.Session() :
    x = tf.get_variable(name='fm_v', shape=[10, 4], trainable=True, initializer= tf.constant_initializer(np.array([0,1])))
    #x = tf.get_variable('x', shape = [10, 4], initializer = tf.constant_initializer(value), trainable=True)
    x.initializer.run()
    print(x.eval())

fitting shape:
[[0. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
